In [1]:
!wget -O data.zip "https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/4drtyfjtfy-1.zip"

--2022-02-18 10:05:24--  https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/4drtyfjtfy-1.zip
Resolving md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)... 52.218.120.106
Connecting to md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)|52.218.120.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95592747 (91M) [application/octet-stream]
Saving to: ‘data.zip’

data.zip            100%[===================>]  91.16M  25.3MB/s    in 3.6s    

2022-02-18 10:05:28 (25.3 MB/s) - ‘data.zip’ saved [95592747/95592747]



In [2]:
!unzip data.zip

Archive:  data.zip
  inflating: dataset2.zip            


In [3]:
!unzip -q dataset2.zip

In [4]:
import glob, os, shutil

In [5]:
types = [
  ("Cloudy", "cloud*.*"), ("Rainy" , "rain*.*"), ("Shiny", "shin*.*"), ("Sunrise","sunrise*.*")       
]

shutil.rmtree("dataset", ignore_errors=True)
for folder_name, file_mask in types:
  os.makedirs(f"dataset/Train/{folder_name}")
  os.makedirs(f"dataset/Test/{folder_name}")
  files = sorted( glob.glob(f"dataset2/{file_mask}") )
  train_qnty = int(0.8*len(files))
    
  [ shutil.copy(file, f"dataset/Train/{folder_name}")  for file in files[:train_qnty]  ]
  [ shutil.copy(file, f"dataset/Test/{folder_name}")  for file in files[train_qnty:]  ]
  print(folder_name, file_mask, len(files))

Cloudy cloud*.* 300
Rainy rain*.* 215
Shiny shin*.* 253
Sunrise sunrise*.* 357


In [6]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing.image import load_img, img_to_array

In [7]:
!wget -O cup.jpg "https://lh5.googleusercontent.com/sTDIn5BK6czLJsB9u-m-VcLma4a6IbJ6rKL0NC3Vp8XESJuAoc5detOT9wtNobOX2CvCX4Qtel21vr7R3Jnv13MrbNIy-yhb-4z6H08wIthMXhce8mjglvufpLOzgx0ftFbvGphn"

--2022-02-18 10:32:40--  https://lh5.googleusercontent.com/sTDIn5BK6czLJsB9u-m-VcLma4a6IbJ6rKL0NC3Vp8XESJuAoc5detOT9wtNobOX2CvCX4Qtel21vr7R3Jnv13MrbNIy-yhb-4z6H08wIthMXhce8mjglvufpLOzgx0ftFbvGphn
Resolving lh5.googleusercontent.com (lh5.googleusercontent.com)... 173.194.216.132, 2607:f8b0:400c:c12::84
Connecting to lh5.googleusercontent.com (lh5.googleusercontent.com)|173.194.216.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30838 (30K) [image/jpeg]
Saving to: ‘cup.jpg’

cup.jpg             100%[===================>]  30.12K  --.-KB/s    in 0s      

2022-02-18 10:32:40 (69.9 MB/s) - ‘cup.jpg’ saved [30838/30838]



In [8]:
img = load_img("cup.jpg", target_size=(224,224))
x = img_to_array(img)
x = x.reshape( (1,)+x.shape  )
x.shape

(1, 224, 224, 3)

In [9]:
# inicjalizacja sieci
model = VGG16()
predict = model.predict( preprocess_input(x) )

553476096/553467096 [==============================] - 3s 0us/step


In [11]:
decode_predictions(predict, top=10)

[[('n03063599', 'coffee_mug', 0.25329998),
  ('n07930864', 'cup', 0.1709395),
  ('n04263257', 'soup_bowl', 0.16099006),
  ('n07920052', 'espresso', 0.14152731),
  ('n03733805', 'measuring_cup', 0.08204568),
  ('n04398044', 'teapot', 0.047222648),
  ('n03775546', 'mixing_bowl', 0.044552106),
  ('n03786901', 'mortar', 0.024030209),
  ('n03950228', 'pitcher', 0.01415348),
  ('n04332243', 'strainer', 0.010426581)]]